Module importing

In [11]:
import sys
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset

sys.path.append('/workspace/my_tool/model')
from CNN_Auto import CNNA
sys.path.append('/workspace/my_tool/tool')
from model_builder import k_fold_training
from performance import performance
sys.path.append('/workspace/my_tool/tool/data_load')
from cnn_auto_load import ImageDataset

Data prepairing

In [12]:
#Import_data
dataset = ImageDataset('/workspace/data/results/raw/Ko đạt(366_367G)/images')
index = random.randint(0, len(dataset)-1)

N = len(dataset)
test_len = int(N*0.1)
train_len = N-test_len

train_data, test_data = random_split(dataset, [train_len, test_len])
print(len(test_data))
print(len(dataset))

16
160


Training

In [13]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("cuda available")
else:
    device = torch.device('cpu')
    

para = {"BottleNeck":       1, # an array (1,l) or a number
		    "Structure":        [3,8,16,32,32], # an array (1,N)
				"Encode acfnc":    	"Tanh", # a string
				"Decode acfnc":     "Tanh", # a string
				"Kernel size":      [(4,4),(4,4),(2,2),(2,2),(2,2),(2,2),(4,4),(4,4)], # an array (1,N-1)
				"Stride":           [(2,2),(2,2),(2,2),(2,2),(2,2),(2,2),(2,2),(2,2)], # an array (1,N-1)
				"Padding":          [(1,1),(1,1),(0,0),(0,0),(0,0),(0,0),(1,1),(1,1)], # an array (1,N-1)
				"Pooling":          ["Average",[3,1,1]]
    }


model = CNNA(para)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
# builder = k_fold_training(train_data,1,criterion = nn.L1Loss())
# builder.training(model,device,optimizer,num_epochs=30,batch=8,visualize=1)

train_loss = 0
val_loss = 0

data = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)
loss_fn = nn.L1Loss()
for inputs, labels in data:
    inputs = inputs.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model.forward(inputs)
    loss = loss_fn(outputs,labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.cpu().detach().numpy()
train_loss = train_loss / len(data)


cuda available


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

Performance

In [ ]:
shower = performance()
shower.test(model,device,test_data,per_fnc = "series")
criterion = nn.L1Loss()
for i in range(16):
    output = torch.stack([test_data[i][1]]).to(device)
    input = torch.stack([test_data[i][0]]).to(device)
    y_hat = model.forward(input)
    loss = criterion(y_hat,output)
    out_img = y_hat[0].permute(1, 2, 0).detach().numpy()
    in_img = input[0].permute(1, 2, 0).detach().numpy()
    # Display the image
    fig, axs = plt.subplots(1, 2, figsize=(6, 10))  # 2 rows, 1 column

    # Display the first image
    axs[0].imshow(in_img)
    axs[0].axis('off')                # Hide axis
    axs[0].set_title('Image 1')       # Optional title

    # Display the second image
    axs[1].imshow(out_img)
    axs[1].axis('off')
    axs[1].set_title('Image 2')

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Show the plot
    plt.show()
    print(loss.cpu().detach().numpy())